# Central Hidroeléctrica Yacyretá
## 01 - Lectura de los datos y armado del Dataset
### Fecha: 28-Nov-21

In [1]:
# Cargo numpy y pandas para trabajar con dataframes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

In [2]:
# Create DataFrame with a single date column  
CHY = pd.DataFrame({'FECHA': pd.date_range('2010-01-01', '2021-12-31')}) 

In [3]:
CHY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4383 entries, 0 to 4382
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FECHA   4383 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 34.4 KB


In [4]:
CHY

,FECHA
0,2010-01-01
1,2010-01-02
2,2010-01-03
3,2010-01-04
4,2010-01-05
...,...
4378,2021-12-27
4379,2021-12-28
4380,2021-12-29
4381,2021-12-30


In [5]:
# Check all dates are distinct
CHY.FECHA.nunique()

4383

## Cotas de Embalse y de Restitución

In [6]:
# Especifico el nombre del archivo y el path
path = './datasets/CHY/'
file_name = 'Cota Embalse y Restitución.xlsx'

In [7]:
# Vuelvo a leer el archivo excel, especificando las filas a saltar y las columnas a leer 
df = pd.read_excel(path + file_name, 
                   sheet_name = 'Cotas',
                   skiprows = range(0, 2),
                   usecols = 'C:E')

# Renombro algunas columnas
df = df.rename(columns={'HEMBC [msnm]': 'COTA_EMBALSE', 'HRES [msnm]': 'COTA_RESTITUCION'})

In [8]:
df

,FECHA,COTA_EMBALSE,COTA_RESTITUCION
0,2011-01-01,82.04,60.17
1,2011-01-02,82.06,60.17
2,2011-01-03,82.07,60.12
3,2011-01-04,82.08,60.12
4,2011-01-05,82.13,60.08
...,...,...,...
3850,2021-07-17,82.86,58.96
3851,2021-07-18,82.86,59.01
3852,2021-07-19,82.88,59.01
3853,2021-07-20,82.88,59.01


In [9]:
#CHY = pd.merge(CHY_df, df, how='left', on='FECHA', validate='one_to_one', indicator=True)

In [10]:
CHY = CHY.merge(df, how='left', on='FECHA', validate='one_to_one', indicator=True)

In [11]:
CHY

,FECHA,COTA_EMBALSE,COTA_RESTITUCION,_merge
0,2010-01-01,NaN,NaN,left_only
1,2010-01-02,NaN,NaN,left_only
2,2010-01-03,NaN,NaN,left_only
3,2010-01-04,NaN,NaN,left_only
4,2010-01-05,NaN,NaN,left_only
...,...,...,...,...
4378,2021-12-27,NaN,NaN,left_only
4379,2021-12-28,NaN,NaN,left_only
4380,2021-12-29,NaN,NaN,left_only
4381,2021-12-30,NaN,NaN,left_only


In [12]:
CHY._merge.value_counts()

both          3855
left_only      528
right_only       0
Name: _merge, dtype: int64

In [13]:
# Elimino las columna _merge
CHY=CHY.drop(columns=['_merge'])

In [14]:
# Calculo la columna SALTO
CHY['SALTO'] = CHY['COTA_EMBALSE'] - CHY['COTA_RESTITUCION']

In [15]:
CHY

,FECHA,COTA_EMBALSE,COTA_RESTITUCION,SALTO
0,2010-01-01,NaN,NaN,NaN
1,2010-01-02,NaN,NaN,NaN
2,2010-01-03,NaN,NaN,NaN
3,2010-01-04,NaN,NaN,NaN
4,2010-01-05,NaN,NaN,NaN
...,...,...,...,...
4378,2021-12-27,NaN,NaN,NaN
4379,2021-12-28,NaN,NaN,NaN
4380,2021-12-29,NaN,NaN,NaN
4381,2021-12-30,NaN,NaN,NaN


## Caudal Afluente

In [16]:
# Especifico el nombre del archivo y el path
#path = './datasets/CHY/'

file_name = 'Caudal Afluente.xlsx'

In [17]:
# Leo el archivo, descartando filas y columnas en blanco
df = pd.read_excel(path + file_name, header=2).dropna(axis='columns', how='all').dropna(axis='rows', how='all')
# Ajusto el nombre de la columna de caudal afluente
df.rename(columns={'QAFLR [m3/s]': 'QAFLR'}, inplace = True)
#df

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3855 entries, 0 to 3854
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FECHA   3855 non-null   datetime64[ns]
 1   QAFLR   3855 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 90.4 KB


In [19]:
df

,FECHA,QAFLR
0,2011-01-01,13936.0
1,2011-01-02,13267.0
2,2011-01-03,12670.0
3,2011-01-04,12909.0
4,2011-01-05,13787.0
...,...,...
3850,2021-07-17,6657.0
3851,2021-07-18,6520.0
3852,2021-07-19,6513.0
3853,2021-07-20,5962.0


In [20]:
CHY = CHY.merge(df, how='left', on='FECHA', validate='one_to_one', indicator=True)

In [21]:
CHY._merge.value_counts()

both          3855
left_only      528
right_only       0
Name: _merge, dtype: int64

In [22]:
CHY

,FECHA,COTA_EMBALSE,COTA_RESTITUCION,SALTO,QAFLR,_merge
0,2010-01-01,NaN,NaN,NaN,NaN,left_only
1,2010-01-02,NaN,NaN,NaN,NaN,left_only
2,2010-01-03,NaN,NaN,NaN,NaN,left_only
3,2010-01-04,NaN,NaN,NaN,NaN,left_only
4,2010-01-05,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...
4378,2021-12-27,NaN,NaN,NaN,NaN,left_only
4379,2021-12-28,NaN,NaN,NaN,NaN,left_only
4380,2021-12-29,NaN,NaN,NaN,NaN,left_only
4381,2021-12-30,NaN,NaN,NaN,NaN,left_only


In [23]:
# Elimino las columna _merge
CHY.drop(columns=['_merge'], inplace=True)

In [24]:
CHY

,FECHA,COTA_EMBALSE,COTA_RESTITUCION,SALTO,QAFLR
0,2010-01-01,NaN,NaN,NaN,NaN
1,2010-01-02,NaN,NaN,NaN,NaN
2,2010-01-03,NaN,NaN,NaN,NaN
3,2010-01-04,NaN,NaN,NaN,NaN
4,2010-01-05,NaN,NaN,NaN,NaN
...,...,...,...,...,...
4378,2021-12-27,NaN,NaN,NaN,NaN
4379,2021-12-28,NaN,NaN,NaN,NaN
4380,2021-12-29,NaN,NaN,NaN,NaN
4381,2021-12-30,NaN,NaN,NaN,NaN


## Caudales Turbinado y Vertidos

In [25]:
# Especifico el nombre del archivo y el path
#path = './datasets/CHY/'

file_name = 'Caudal Turbinado y Vertido.xlsx'

In [26]:
# Leo el archivo, descartando filas y columnas en blanco
df = pd.read_excel(path + file_name, header=2).dropna(axis='columns', how='all').dropna(axis='rows', how='all')
df

,FECHA,QVBP [m3/s],QVBAC [m3/s],QTURB [m3/s]
0,2011-01-01,0,1045,11977
1,2011-01-02,0,1045,11863
2,2011-01-03,0,1045,11439
3,2011-01-04,0,1045,11505
4,2011-01-05,0,1047,11455
...,...,...,...,...
3850,2021-07-17,0,1018,5268
3851,2021-07-18,0,1019,5501
3852,2021-07-19,0,1019,5494
3853,2021-07-20,0,1019,5494


In [27]:
# Veo información básica
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3855 entries, 0 to 3854
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   FECHA         3855 non-null   datetime64[ns]
 1   QVBP [m3/s]   3855 non-null   int64         
 2   QVBAC [m3/s]  3855 non-null   int64         
 3   QTURB [m3/s]  3855 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 150.6 KB


In [28]:
# Verifico duplicados
df.duplicated(subset='FECHA').value_counts()

False    3855
dtype: int64

In [29]:
# Verifico si hay nulos
df.isnull().sum()

FECHA           0
QVBP [m3/s]     0
QVBAC [m3/s]    0
QTURB [m3/s]    0
dtype: int64

In [30]:
# Ajusto el nombre de la columna de caudal afluente
df.rename(columns={'QVBP [m3/s]':'QVBP', 'QVBAC [m3/s]':'QVBAC', 'QTURB [m3/s]':'QTURB' }, inplace = True)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3855 entries, 0 to 3854
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FECHA   3855 non-null   datetime64[ns]
 1   QVBP    3855 non-null   int64         
 2   QVBAC   3855 non-null   int64         
 3   QTURB   3855 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 150.6 KB


In [32]:
CHY = CHY.merge(df, how='left', on='FECHA', validate='one_to_one', indicator=True)

In [33]:
CHY._merge.value_counts()

both          3855
left_only      528
right_only       0
Name: _merge, dtype: int64

In [34]:
# Elimino las columna _merge
CHY.drop(columns=['_merge'], inplace=True)

In [35]:
CHY

,FECHA,COTA_EMBALSE,COTA_RESTITUCION,SALTO,QAFLR,QVBP,QVBAC,QTURB
0,2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4378,2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379,2021-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4380,2021-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4381,2021-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Verifico si hay nulos
CHY.isnull().sum()

FECHA                 0
COTA_EMBALSE        528
COTA_RESTITUCION    528
SALTO               528
QAFLR               528
QVBP                528
QVBAC               528
QTURB               528
dtype: int64

In [37]:
# Elimino esas filas con nulos
CHY.dropna(subset=['COTA_EMBALSE', 'COTA_RESTITUCION', 'QAFLR', 'QVBP', 'QVBAC', 'QTURB'], inplace=True)

In [38]:
# Verifico si hay nulos
CHY.isnull().sum()

FECHA               0
COTA_EMBALSE        0
COTA_RESTITUCION    0
SALTO               0
QAFLR               0
QVBP                0
QVBAC               0
QTURB               0
dtype: int64

In [39]:
CHY

,FECHA,COTA_EMBALSE,COTA_RESTITUCION,SALTO,QAFLR,QVBP,QVBAC,QTURB
365,2011-01-01,82.04,60.17,21.87,13936.0,0.0,1045.0,11977.0
366,2011-01-02,82.06,60.17,21.89,13267.0,0.0,1045.0,11863.0
367,2011-01-03,82.07,60.12,21.95,12670.0,0.0,1045.0,11439.0
368,2011-01-04,82.08,60.12,21.96,12909.0,0.0,1045.0,11505.0
369,2011-01-05,82.13,60.08,22.05,13787.0,0.0,1047.0,11455.0
...,...,...,...,...,...,...,...,...
4215,2021-07-17,82.86,58.96,23.90,6657.0,0.0,1018.0,5268.0
4216,2021-07-18,82.86,59.01,23.85,6520.0,0.0,1019.0,5501.0
4217,2021-07-19,82.88,59.01,23.87,6513.0,0.0,1019.0,5494.0
4218,2021-07-20,82.88,59.01,23.87,5962.0,0.0,1019.0,5494.0


In [40]:
# Rango de fechas
print('\nRango de fechas dataset (min / max): ', CHY.FECHA.min(), ' - ', CHY.FECHA.max())


Rango de fechas dataset (min / max):  2011-01-01 00:00:00  -  2021-07-21 00:00:00


## Energía Generada (Argentina y Paraguay)

In [41]:
# Especifico el nombre del archivo y el path
#path = './datasets/CHY/'

file_name = 'Energía Suministrada.xls'

In [42]:
# Leo el archivo, descartando filas y columnas en blanco
df = pd.read_excel(path + file_name, header=1).dropna(axis='columns', how='all').dropna(axis='rows', how='all')

# Echo un vistazo al df leido
df.to_csv('energia_generada.csv')

In [43]:
# Veo las columnas que contiene
df.columns

Index(['Fecha (Diario)', 'Energía SADI (Argentina) [MWh]',
       'Energía SINP (Paraguay) [MWh]', 'Fecha (Diario).1',
       'Energía SADI (Argentina) [MWh].1', 'Energía SINP (Paraguay) [MWh].1',
       'Fecha (Diario).2', 'Energía SADI (Argentina) [MWh].2',
       'Energía SINP (Paraguay) [MWh].2', 'Fecha (Diario).3',
       'Energía SADI (Argentina) [MWh].3', 'Energía SINP (Paraguay) [MWh].3',
       'Fecha (Diario).4', 'Energía SADI (Argentina) [MWh].4',
       'Energía SINP (Paraguay) [MWh].4', 'Fecha (Diario).5',
       'Energía SADI (Argentina) [MWh].5', 'Energía SINP (Paraguay) [MWh].5',
       'Fecha (Diario).6', 'Energía SADI (Argentina) [MWh].6',
       'Energía SINP (Paraguay) [MWh].6', 'Fecha (Diario).7',
       'Energía SADI (Argentina) [MWh].7', 'Energía SINP (Paraguay) [MWh].7',
       'Fecha (Diario).8', 'Energía SADI (Argentina) [MWh].8',
       'Energía SINP (Paraguay) [MWh].8', 'Fecha (Diario).9',
       'Energía SADI (Argentina) [MWh].9', 'Energía SINP (Paraguay) [

In [44]:
# Veo las columnas que comienzan con 'Fecha'
columnas_fecha = [col for col in df.columns if col.startswith('Fecha')]
columnas_fecha

['Fecha (Diario)',
 'Fecha (Diario).1',
 'Fecha (Diario).2',
 'Fecha (Diario).3',
 'Fecha (Diario).4',
 'Fecha (Diario).5',
 'Fecha (Diario).6',
 'Fecha (Diario).7',
 'Fecha (Diario).8',
 'Fecha (Diario).9',
 'Fecha (Diario).10',
 'Fecha (Diario).11']

In [45]:
# Apilo las columnas de fecha y borro las columnas que no necesito
energia_df = df[columnas_fecha].stack().reset_index()
energia_df.drop(columns=['level_0', 'level_1'], inplace = True)
energia_df.rename(columns={0: 'FECHA'}, inplace = True)

In [46]:
# Rango de fechas
print('\nRango de fechas, energía generada Argentina y Paraguay: ', energia_df.FECHA.min(), ' - ', energia_df.FECHA.max())


Rango de fechas, energía generada Argentina y Paraguay:  2011-01-01 00:00:00  -  2021-06-30 00:00:00


In [47]:
# Información de la serie
energia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3833 entries, 0 to 3832
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FECHA   3833 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 30.1 KB


In [48]:
# Verifico duplicados
energia_df.duplicated(subset='FECHA').value_counts()

False    3833
dtype: int64

In [49]:
# Energía generada lado Paraguay
columnas_energia_SINP = [col for col in df.columns if col.startswith('Energía SINP')]
#columnas_energia_SINP

In [50]:
# Agrego esta columna al df
energia_df['SINP'] = df[columnas_energia_SINP].stack().reset_index()[0]

In [51]:
# Energía generada lado Argentina
columnas_energia_SADI = [col for col in df.columns if col.startswith('Energía SADI')]
#columnas_energia_SADI

In [52]:
# Agrego esta columna al df
energia_df['SADI'] = df[columnas_energia_SADI].stack().reset_index()[0]
#energia_df

In [53]:
# Ordeno el df por la columna FECHA
energia_df.sort_values('FECHA', inplace=True)

In [54]:
energia_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3833 entries, 0 to 3829
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FECHA   3833 non-null   datetime64[ns]
 1   SINP    3833 non-null   float64       
 2   SADI    3833 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 119.8 KB


In [55]:
#Agrego los datos de la energía generada al dataset general
CHY = CHY.merge(energia_df, how='left', on='FECHA', validate='one_to_one', indicator=True)

In [56]:
# Borro las filas en las cuales las energías son ambas nullas
CHY.dropna(subset=['SINP', 'SADI'], inplace=True)

In [57]:
# Borro la columna de informacikón _null
CHY.drop(columns=['_merge'], inplace=True)

In [58]:
# Verifico si quedaron nulos
CHY.isnull().sum()

FECHA               0
COTA_EMBALSE        0
COTA_RESTITUCION    0
SALTO               0
QAFLR               0
QVBP                0
QVBAC               0
QTURB               0
SINP                0
SADI                0
dtype: int64

In [59]:
# Rango de fechas
print('\nRango de fechas general: ', CHY.FECHA.min(), ' - ', CHY.FECHA.max())


Rango de fechas general:  2011-01-01 00:00:00  -  2021-06-30 00:00:00


In [61]:
# Grabo el dataset a un csv
CHY.to_csv(path + '\output\' + 'CHY_dataset.csv', index=False)

# ¡Fin del Notebook!